# `static-typing` examples

## Static type information in AST

In [1]:
import typing as t
import static_typing as st

code = '''

import typing as t

spam = ['spam', 'spam', 'spam']  # type: t.List[str]

class MyClass:
    def __init__(self):
        self.a = 42  # type: int
        self.b = 42.0  # type: float

def add_one(n: int) -> int:
    temp = n + 1  # type: int
    return temp

'''

module = st.parse(code)

In [2]:
module._module_vars

{'spam': OrderedSet([typing.List[str]])}

In [3]:
module._classes

{'MyClass': <StaticallyTypedClassDefClass@4463431408>}

In [4]:
cls = module._classes['MyClass']
cls._instance_fields

{'a': OrderedSet([<class 'int'>]), 'b': OrderedSet([<class 'float'>])}

In [5]:
cls._methods

{'__init__': <StaticallyTypedFunctionDefClass@4463431576>}

In [6]:
method = cls._methods['__init__']
method._kind

<FunctionKind.Constructor: 7>

In [7]:
method._nonlocal_assignments

{<_ast3.Attribute at 0x10a0a8fd0>: OrderedSet([<class 'int'>]),
 <_ast3.Attribute at 0x10a0ba0f0>: OrderedSet([<class 'float'>])}

In [8]:
module._functions

{'add_one': <StaticallyTypedFunctionDefClass@4463503288>}

In [9]:
function = module._functions['add_one']
function._params

{'n': OrderedSet([<class 'int'>])}

In [10]:
function._returns

OrderedSet([<class 'int'>])

In [11]:
function._kind

<FunctionKind.Function: 1>

In [12]:
function._local_vars

{'temp': OrderedSet([<class 'int'>])}

## Other features

### AST validation

In [13]:
import ast
import static_typing as st

validator = st.ast_manipulation.AstValidator[ast]()

tree = ast.parse(code).body[3]

validator.visit(tree)

Let's make the AST invalid by adding something unexpected.

In [14]:
tree.body.append(ast.Slice(ast.Num(-10), ast.Num(10), ast.Num(2)))

In [15]:
try:
    validator.visit(tree)
except AssertionError:
    print('tree is invalid')

tree is invalid


We can still unparse that AST...

In [16]:
import astunparse

print(astunparse.unparse(tree))



def add_one(n: int) -> int:
    temp = (n + 1)
    return temp-10:10:2



But the resulting code is invalid because unparser assumes 100% valid AST!

### AST transcribing

In [17]:
import ast
import typed_ast.ast3
import static_typing as st

transcriber = st.ast_manipulation.AstTranscriber[typed_ast.ast3, ast]()

typed_tree = typed_ast.ast3.parse(code).body[3]

typed_tree, typed_tree.args

(<_ast3.FunctionDef at 0x10a121438>, <_ast3.arguments at 0x10a1300b8>)

In [18]:
tree = transcriber.visit(typed_tree)

tree, tree.args

(<_ast.FunctionDef at 0x10a121048>, <_ast.arguments at 0x10a1211d0>)

In [19]:
ast.dump(tree.args)

"arguments(args=[arg(arg='n', annotation=Name(id='int', ctx=Load()))], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[])"